## Things you need
+ config file with api key
+ Your user agent info can get https://www.whatismybrowser.com/detect/what-is-my-user-agent and replace the headers variable with your info
    + header variable is line 10 in both scraping block
+ Script splits up your sources into 2 queries and saves them as csv files.  I created a csv folder for them

## change query
Hit other candidates as well and make sure that the range (20,31) is no more than 30 days ago.  If you get a out of range error just change 20 to whatever day of month it is, or day of month +1

In [1]:
import pandas as pd
import config
import math
import csv
import sys
from newsapi import NewsApiClient
import newspaper
import requests
from newspaper import fulltext
import time
# Hit Api with credentials
newsapi = NewsApiClient(api_key=config.api_key)


## global variables
candidates_list = []
total_count = []


def error_checks():
    ##validate that df shape[0] is equal to expected query count
    print("we should expect: {} articles".format(sum(total_count)))
    print("we have: {} articles".format(Cand_df.shape[0]))


    ## test 100 results theory
    over_100 = sum([x-100 for x in total_count if x>100 ])
    print("{} results came into query that exceeded 100 hits in a day.".format(over_100))


    ##unique links
    unique_array = Cand_df.url.unique()
    print("we have {} unique links".format(unique_array.shape[0]))
    
    duplicateRowsDF = Cand_df[Cand_df.duplicated("url")]
    print("we have duplciate shape df of: {} ".format(duplicateRowsDF.shape))

def clean_query(query):
    for x in query['articles']:
        try:
            x["source"] = x["source"]["name"]
        except:
            pass
        try:
            x['publishedAt'] = str.split(x['publishedAt'], "T")[0]
        except:
            pass
        try:
            del x['urlToImage']
        except KeyError:
            pass
    my_df = pd.DataFrame(query["articles"])
    return my_df

def hit_api(start, end, q, myStr):

    # catch bug with formatted strings for dates
    if end < 10:
        start_str = "0" + str(start)
        end_str = "0" + str(end)
    elif end == 10:
        start_str = "0" + str(start)
        end_str = str(end)
    else:
        start_str = str(start)
        end_str = str(end)

    # API query
    all_articles = newsapi.get_everything(q=q,
                                          sources=myStr,
                                          language='en',
                                          from_param='2019-10-{}'.format(
                                              start_str),
                                          to='2019-10-{}'.format(end_str),
                                          sort_by='relevancy',
                                          page_size=100,
                                          page=1)

    # get count
    total_pages = math.ceil(all_articles["totalResults"]/100)
    print("query will return: " + str(all_articles["totalResults"]))

    # store count to check versus dimension of df later
    total_count.append(all_articles["totalResults"])

    # Clen query
    all_articles = clean_query(all_articles)

    # append to list
    candidates_list.append(all_articles)
    return(candidates_list)


In [2]:

## import data dictionary form github
url = 'https://raw.githubusercontent.com/mburke65/Data_698_Final/master/lists.csv'
df = pd.read_csv(url, error_bad_lines=False)

# picking data sources from file on github
a= df.iloc[[16,17,24],:]
b= df.iloc[[40,43,77],:]
list_sources = a["sources"].tolist()

# build out string for query request
myString = ",".join(list_sources)

list_sources2 = b["sources"].tolist()
myString2 = ",".join(list_sources2)



## Scrape Wall street journal and NYT

In [5]:

for x in range(3, 31):
    df = hit_api(x, x+1, 'Trump OR Donald Trump', myString2)

# collapse list on itself to build big df
Cand_df = pd.concat(df)
Cand_df = Cand_df.reset_index(drop=True)
error_checks()


headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'}
## Build counts as check on progress of loop. will count down Url left to hit
counts=Cand_df['url'].shape[0]
list_full_text=[]
for link in Cand_df['url']:
    counts-=1
    print(counts)
    html = requests.get(link,headers=headers).text
    try: text = fulltext(html)
    except: 
        print("no words found")
        text="no words found"
        list_full_text.append(text)
        continue
    list_full_text.append(text)
Cand_df['full_art']=pd.Series(list_full_text)
Cand_df.to_csv('data_csvs/data1'+str(time.time())+'.csv')

query will return: 288
query will return: 200
query will return: 92
query will return: 142
query will return: 240
query will return: 278
query will return: 299
query will return: 284
query will return: 184
query will return: 87
query will return: 118
query will return: 191
query will return: 238
query will return: 271
query will return: 243
query will return: 170
query will return: 93
query will return: 121
query will return: 229
query will return: 283
query will return: 280
query will return: 241
query will return: 161
query will return: 90
query will return: 138
query will return: 217
query will return: 252
query will return: 257
we should expect: 5687 articles
we have: 2762 articles
2925 results came into query that exceeded 100 hits in a day.
we have 1616 unique links
we have duplciate shape df of: (1146, 7) 
2761
2760
2759
2758
2757
2756
2755
2754
2753
2752
2751
2750
2749
2748
2747
2746
2745
2744
2743
2742
2741
2740
2739
2738
2737
2736
2735
2734
2733
2732
2731
2730
2729
2728
2727


1440
1439
1438
1437
1436
1435
1434
1433
1432
1431
1430
1429
1428
1427
1426
1425
1424
1423
1422
1421
1420
1419
1418
1417
1416
1415
1414
1413
1412
1411
1410
1409
1408
1407
no words found
1406
1405
1404
1403
1402
1401
1400
1399
1398
1397
1396
1395
no words found
1394
1393
1392
1391
1390
1389
1388
no words found
1387
1386
no words found
1385
1384
1383
1382
1381
1380
1379
1378
1377
1376
1375
1374
1373
1372
1371
1370
1369
1368
1367
1366
1365
1364
1363
1362
1361
1360
1359
1358
1357
1356
1355
1354
1353
1352
1351
1350
1349
1348
1347
1346
1345
1344
1343
1342
1341
1340
1339
1338
1337
1336
1335
1334
1333
1332
1331
1330
1329
1328
1327
1326
1325
1324
1323
1322
1321
1320
1319
1318
1317
1316
1315
1314
1313
1312
1311
1310
1309
1308
1307
1306
1305
1304
1303
1302
1301
1300
1299
1298
1297
1296
1295
no words found
1294
1293
1292
1291
1290
1289
no words found
1288
1287
1286
1285
1284
1283
1282
1281
1280
1279
1278
1277
1276
1275
1274
1273
1272
1271
1270
1269
1268
1267
1266
1265
1264
1263
1262
1261
1260
1259


In [ ]:
Cand_df['full_art']=pd.Series(list_full_text)
Cand_df.to_csv('data_csvs/data1'+str(time.time())+'.csv')

## Scrape WAPO New York magazine


In [ ]:
Cand_df=None

for x in range(1, 29):
    df = hit_api(x, x+1, 'Trump OR Donald Trump', myString)

# collapse list on itself to build big df
Cand_df = pd.concat(df)
Cand_df = Cand_df.reset_index(drop=True)
error_checks()


## Build counts as check on progress of loop. will count down Url left to hit
counts=Cand_df['url'].shape[0]
list_full_text=[]
for link in Cand_df['url']:
    counts-=1
    print(counts)
    html = requests.get(link,headers=headers).text
    try: text = fulltext(html)
    except: 
        print("no words found")
        text="no words found"
        list_full_text.append(text)
        continue
    list_full_text.append(text)
Cand_df['full_art']=pd.Series(list_full_text)
Cand_df.to_csv('data_csvs/data1'+str(time.time())+'.csv')

In [ ]:
my_df=pd.read_csv('Vikas.csv')

In [ ]:
my_df.columns

In [ ]:
my_df["full_art"]

In [ ]:
Cand_df['full_art']